In [15]:
import os

In [18]:
assert os.path.isfile('hn_data/hn.csv'), "HN Data missing, check the README"

In [10]:
import pandas as pd
import numpy as np

# Plotting
import pygal
import seaborn as sns
import matplotlib.pyplot as plt

import calendar

from utils import render

%matplotlib inline

In [22]:
df = pd.read_csv(
    'hn_data/hn.csv', index_col='Object ID',
    parse_dates=['Created At'],
    dtype={
        'Post Type': 'category'
    })

In [23]:
df.head()

,Title,Post Type,Author,Created At,URL,Points,Number of Comments
Object ID,,,,,,,
1,Y Combinator,story,pg,2006-10-09 18:21:51,http://ycombinator.com,61,18.0
2,A Student's Guide to Startups,story,phyllis,2006-10-09 18:30:28,http://www.paulgraham.com/mit.html,16,1.0
3,Woz Interview: the early days of Apple,story,phyllis,2006-10-09 18:40:33,http://www.foundersatwork.com/stevewozniak.html,7,1.0
4,NYC Developer Dilemma,story,onebeerdave,2006-10-09 18:47:42,http://avc.blogs.com/a_vc/2006/10/the_nyc_deve...,5,1.0
5,"Google, YouTube acquisition announcement could...",story,perler,2006-10-09 18:51:04,http://www.techcrunch.com/2006/10/09/google-yo...,7,1.0


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2833355 entries, 1 to 21529222
Data columns (total 7 columns):
Title                 object
Post Type             category
Author                object
Created At            datetime64[ns]
URL                   object
Points                int64
Number of Comments    float64
dtypes: category(1), datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 154.0+ MB


##### Total posts:

In [25]:
f"{df.shape[0]:,}"

'2,833,355'

##### Posts per type:

In [26]:
df['Post Type'].value_counts()

story      2647714
ask_hn      107370
show_hn      76726
poll          1545
Name: Post Type, dtype: int64

In [27]:
chart = pygal.Pie(inner_radius=.4, half_pie=True, explicit_size=600)
chart.title = 'Post type share'
for title, count in df['Post Type'].value_counts().items():
    chart.add(title, count)
render(chart)

### Data Preparation

In my analysis I want to compare the evolution of post types per different date ranges. For example, per month, day of the week, or hour of the day. I'll do the minimum set of data preparation to help me with the analysis and visualizations later.

##### 1) Localizing `Created At` to Eastern
The `Created At` column contains a datetime with the post created, in UTC. We must localize it using pandas and `pytz`:

In [28]:
df['Created At Eastern'] = df['Created At'].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

##### 2) Creating a Monthly `Period` for easy search
Simplifying access for later queries extracting only the monthly `Period` of a post. If a post was created on `2019-10-31T00:56:23 EST`, the result will be `Period(2019-10)`.

In [29]:
# Old school: df['created_at_et'].apply(lambda val: pd.Timestamp(year=val.year, month=val.month, day=1))
df['Rounded Month'] = df['Created At Eastern'].dt.to_period('m')

##### 3) Extracting day of week and hour of day

Extracting what day of the week and at what hour the post was created.

In [30]:
day_conversion_table = {k: calendar.day_name[k] for k in range(7)}
day_conversion_table

{0: 'Monday',
 1: 'Tuesday',
 2: 'Wednesday',
 3: 'Thursday',
 4: 'Friday',
 5: 'Saturday',
 6: 'Sunday'}

In [31]:
df['Day of Week'] = df['Created At Eastern'].dt.dayofweek.replace(day_conversion_table)

In [32]:
df['Hour of the Day'] = df['Created At Eastern'].dt.hour

##### 3) Removing the posts from last month

To avoid partial results, we'll consider the last full month.

In [33]:
df.drop(df.loc[df['Rounded Month'] == df['Rounded Month'].iloc[-1]].index, inplace=True)

### Putting everything together

In [83]:
def read_hn_file(filepath, drop_last_month=True):
    df = pd.read_csv(
        filepath, index_col='Object ID',
        parse_dates=['Created At'],
        dtype={
            'Post Type': 'category'
        })
    
    df['Created At Eastern'] = df['Created At'].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
    df['Rounded Month'] = df['Created At Eastern'].dt.to_period('m')
    
    day_conversion_table = {k: calendar.day_name[k] for k in range(7)}
    df['Day of Week'] = df['Created At Eastern'].dt.dayofweek.replace(day_conversion_table)
    df['Hour of the Day'] = df['Created At Eastern'].dt.hour
    
    if drop_last_month:
        df.drop(df.loc[df['Rounded Month'] == df['Rounded Month'].iloc[-1]].index, inplace=True)
    return df